In [10]:
# Importieren der erforderlichen Module
from tkinter import *
from tkinter import messagebox
import random

#Herstellung des Spielfenster/Interface
window = Tk()
window.geometry("650x690")
window.title("Tic-Tac-Toe")

# Spieler und Zufallsauswahl für den Startspieler
players = ["ai", "hu"] 
player = random.choice(players)
aiMark = players[0]
humanMark = players[1]

# Darstellen der UI-Werte und des Spielfelds mit den 9 Buttons
buttons = [[0,0,0],
           [0,0,0],
           [0,0,0]]

board = [
    ["", "", ""],
    ["", "", ""],
    ["", "", ""]]

# Funktion zum Beenden des Spiels    
def exit_game():
    window.destroy()

# Definition einer Funktion, um den Zug der KI am Anfang zu behandeln
def place_x():
    aiTurn()
    
                    
# Definition einer Funktion, um ein neues Spiel zu starten
def new_game():
    global player, count, board
    player = random.choice(players)

    label.config(text=player+"'s turn")
    
    buttons[0][0].config(text="", bg="#F0F0F0")
    buttons[0][1].config(text="", bg="#F0F0F0")
    buttons[0][2].config(text="", bg="#F0F0F0")
    
    buttons[1][0].config(text="", bg="#F0F0F0")
    buttons[1][1].config(text="", bg="#F0F0F0")
    buttons[1][2].config(text="", bg="#F0F0F0")
    
    buttons[2][0].config(text="", bg="#F0F0F0")
    buttons[2][1].config(text="", bg="#F0F0F0")
    buttons[2][2].config(text="", bg="#F0F0F0")
    
    count=0
    
    board = [
    ["", "", ""],
    ["", "", ""],
    ["", "", ""]]

    enable_buttons()
    

# Text für die Benutzeroberfläche
label = Label(text=player + "'s turn", font=('consolas',40))
label.pack(side="top")

# Schaltfläche zum Neustarten des Spiels
reset_button = Button(text="restart", font=('consolas',20), bg="plum1", command=new_game)
reset_button.pack(side="top")

# Schaltfläche zum Beenden des Spiels
exit_button = Button(text="exit", font=('consolas',20), bg="plum1", command=exit_game)
exit_button.pack(side = "top")

# Schaltfläche zum Starten des Zuges der KI
startforai_button = Button(text="start for ai", font=('consolas',20), bg='plum1', command=place_x)
startforai_button.pack(side="left")

frame = Frame(window)
frame.pack()


#Herstellung eines 3x3-Raster von Buttons im frame
for row in range(3):
    for column in range(3):
        buttons[row][column] = Button(frame, text="",font=('consolas',40), width=5, height=2,relief="groove",
                                      command= lambda row=row, column=column: next_turn(row,column))
        buttons[row][column].grid(row=row,column=column)
        
count=0

Tie="Tie"


# depth between 0-9:
depth = 9

# Minimax-Algorithmus für die KI
def minimax(node, depth, maximizingPlayer):
    global counter, buttons
    emptyCells = [(row, column) for row in range(3) for column in range(3) if board[row][column] == ""]
    moves = [(row,column) for row in range(3) for column in range(3) if buttons[row][column]['text'] == ""]
    move = random.choice(moves)
    
    if depth == 0 or is_terminal(node) == True:
        return evaluate(node, depth), move
    
    if maximizingPlayer:
        max_score = float('-inf')
        best_moves = []
        for row,column in emptyCells:
            board[row][column] = aiMark
            score, _ = minimax(board, depth - 1 , False)
            board[row][column] = ""
            if score > max_score:
                max_score = score
                best_moves = [(row, column)]
            elif score == max_score:
                best_moves.append((row, column))
        best_move = random.choice(best_moves)
        return max_score, best_move
        
    else:
        min_score = float('inf')
        best_move = None
        for row,column in emptyCells:
            board[row][column] = humanMark
            score, _ = minimax(board, depth - 1 , True)
            board[row][column] = ""
            if score < min_score:
                min_score = score
                best_move = (row, column)
        return min_score, best_move
        
    

# Bewertung des aktuellen Spielzustands und Zuweisung einer Punktzahl
def evaluate(node, depth): 
    
    if check_winner(node) == 'ai':
            return 100 + depth
    elif check_winner(node) == 'hu':
            return -100 - depth
    elif check_winner(node) == "Tie in node":
            return 0
    else:
        eval = 0
        for row in range(3):
            for column in range (3):
                if node[row][column] == aiMark:
                        if (row == 1 and column == 1):
                            eval += 3  #center
                        if (row == 0 and column == 0) or (row == 0 and column == 2) or (row == 2 and column == 0) or (row == 2 and column == 2):
                            eval += 2  # corners
                        if (row == 0 and column == 1) or (row == 1 and column == 0) or (row == 1 and column == 2) or (row == 2 and column == 1):
                            eval += 1  # Rest of the board

                elif node[row][column] == humanMark:
                        if  (row == 1 and column == 1):
                            eval -= 3 #center
                        if  (row == 0 and column == 0) or (row == 0 and column == 2) or (row == 2 and column == 0) or (row == 2 and column == 2):
                            eval -= 2  # corners
                        if (row == 0 and column == 1) or (row == 1 and column == 0) or (row == 1 and column == 2) or (row == 2 and column == 1):
                            eval -= 1  # Rest of the board

        return eval
    


# Überprüfen, ob das Spiel in einem Endzustand ist
def is_terminal(node):
    if check_winner(node) == 'ai': 
        return True
    if check_winner(node) == 'hu':
        return True
    if check_winner(node) == "Tie in node": 
        return True
    else:
        return False
 
   

# Behandeln des Zugs des KI-Spielers
def aiTurn():
    global count, player, aiMark, check_winner, board
    emptyCells = [(row, column) for row in range(3) for column in range(3) if board[row][column] == "" and buttons[row][column]['state']== NORMAL]

    if emptyCells:
        score, best_move = minimax(board, depth, True)
        row, column = best_move
        buttons[row][column]['text'] = aiMark
        board[row][column] = aiMark  
        
    count+=1 
    if check_winner(board) is False:
        player = players[1]
        label.config(text = player + "'s turn")
    elif check_winner(board) == 'ai':
        label.config(text="Oh no " + player + " wins")
        disable_buttons()
    elif check_winner(board) == "Tie":
        label.config(text="It's a Tie")
        disable_buttons()      
        
# Behandeln des Zugs des menschlichen Spielers
def next_turn(row, column):
    global player, count , aiTurn, place_x, aiPlayed, check_winner
    if player == players[1]:
            if buttons[row][column]['text'] == "":
                buttons[row][column]['text'] = player
                board[row][column] = player
                count += 1
                if check_winner(board) is False:
                    player = players[0] 
                    label.config(text = player + "'s turn")
                    window.after(200, aiTurn)
                elif check_winner(board) == 'hu':
                    label.config(text = "Hurray " + player + " wins")
                    disable_buttons()
                elif check_winner(board) == "Tie":
                    label.config(text = "It's a Tie")
                    disable_buttons()
            else:
                messagebox.showerror(title=None, message='This square is already taken. Please choose another one')
        


def disable_buttons():
    buttons[0][0]['state'] = DISABLED
    buttons[0][1]['state'] = DISABLED
    buttons[0][2]['state'] = DISABLED
    buttons[1][0]['state'] = DISABLED
    buttons[1][1]['state'] = DISABLED
    buttons[1][2]['state'] = DISABLED
    buttons[2][0]['state'] = DISABLED
    buttons[2][1]['state'] = DISABLED
    buttons[2][2]['state'] = DISABLED

    
def enable_buttons():
    buttons[0][0]['state'] = NORMAL
    buttons[0][1]['state'] = NORMAL
    buttons[0][2]['state'] = NORMAL
    buttons[1][0]['state'] = NORMAL
    buttons[1][1]['state'] = NORMAL
    buttons[1][2]['state'] = NORMAL
    buttons[2][0]['state'] = NORMAL
    buttons[2][1]['state'] = NORMAL
    buttons[2][2]['state'] = NORMAL



# Funktion zum Überprüfen des Gewinners
def check_winner(board):
    global count
    
    emptyCells = [(row, column) for row in range(3) for column in range(3) if board[row][column] == ""]
    

    if board[0][0] == board[0][1] == board[0][2] != '':
        if board [0][0] == aiMark:
            return 'ai'
        if board[0][0] == humanMark:
            return 'hu'

    elif board[1][0] == board[1][1] == board[1][2] != '':
        if board[1][0] == aiMark:
            return 'ai'
        if board[1][0] == humanMark:
            return 'hu'

    elif board[2][0] == board[2][1] == board[2][2] != '':
        if board[2][0] == aiMark:
            return 'ai'
        if board[2][0] == humanMark:
            return 'hu'
        
    elif board[0][0] == board[1][0] == board[2][0] != '':
        if board [0][0] == aiMark:
            return 'ai'
        if board[0][0] == humanMark:
            return 'hu'
        
    elif board[0][1] == board[1][1] == board[2][1] != '':
        if board [0][1] == aiMark:
            return 'ai'
        if board[0][1] == humanMark:
            return 'hu'
        
    elif board[0][2] == board[1][2] == board[2][2] != '':
        if board [0][2] == aiMark:
            return 'ai'
        if board[0][2] == humanMark:
            return 'hu'
        
    elif board[2][0] == board[1][1] == board[0][2] != '':
        if board [0][2] == aiMark:
            return 'ai'
        if board[0][2] == humanMark:
            return 'hu'
        
    elif board[0][0] == board[1][1] == board[2][2] != '':
        if board [0][0] == aiMark:
            return 'ai'
        if board[0][0] == humanMark:
            return 'hu'
    
    if count == 9: 
        return "Tie"
    elif not emptyCells:
            return "Tie in node"
    
    else:
        return False
    

        
# Hauptschleife für das Tkinter-Fenster
window.mainloop()